In [18]:
%load_ext autoreload
%autoreload 2

import json
from dwitracts.glm import DwiTractsGlm
import dwitracts.plot as dwiplot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# GLM Analysis using DwiTractsGlm class

config_file = 'project/config_glm_dmn.json'

def _json_object_hook(d): return namedtuple('X', d.keys())(*d.values())
def json2obj(data): return json.loads(data, object_hook=_json_object_hook)

with open(config_file, 'r') as myfile:
    json_string=myfile.read()
    
params = json.loads(json_string)
params_gen = params['general']
params_glm = params['glm']

with open(params_gen['tracts_config_file'], 'r') as myfile:
    json_string=myfile.read()
    
params['tracts'] = json.loads(json_string)

with open(params_gen['preproc_config_file'], 'r') as myfile:
    json_string=myfile.read()
    
params['preproc'] = json.loads(json_string)

my_glm = DwiTractsGlm( params )

# Initialize the object with the current parameters
assert my_glm.initialize()


In [20]:
# Fit all specified GLMs and save results to target directory
assert my_glm.fit_glms( clobber=True, verbose=True )


Computing GLMs for 6 tracts.
Writing results to data/nki/tract_stats/dmn/glms/glm-age-sex.


   Processing PCm_L_PCm_R



Writing residuals (182, 218, 182, 3)


/Users/lpzatr/anaconda3/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/lpzatr/anaconda3/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:325: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/Users/lpzatr/anaconda3/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:256: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


      Wrote glm-age-sex-Intercept
      Wrote glm-age-sex-Age
      Wrote glm-age-sex-Sex
      Wrote glm-age-sex-Age*Sex
   Finished PCm_L_PCm_R
   No tract exists for PCm_L_PFCm_L. Skipping.
   No tract exists for PCm_L_PFCm_R. Skipping.
   No tract exists for PCm_R_PFCm_L. Skipping.
   No tract exists for PCm_R_PFCm_R. Skipping.
   Processing PFCm_L_PFCm_R


/Users/lpzatr/OneDrive - The University of Nottingham/synched/python/github/dwi-tracts/dwitracts/glm.py:263: RuntimeWarning: invalid value encountered in less
  V_tract[V_tract<thres]=0


/Users/lpzatr/anaconda3/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1620: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/lpzatr/anaconda3/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


KeyboardInterrupt: 

In [ ]:
# Extract distance traces from the GLM results
assert my_glm.extract_distance_traces( clobber=True, verbose=True )


In [ ]:
# Extract distance traces with 1D RFT correction from the GLM results
assert my_glm.extract_distance_traces_rft1d( clobber=True, verbose=True, fdr_method='fdr_tsbky', fdr_alpha=0.05 )


In [ ]:
# Plot t-value distance traces

# Font sizes
params['axis_font'] = 24
params['yticklabel_font'] = 12
params['xticklabel_font'] = 20
params['title_font'] = 28

alpha = 0.05

tract_names = my_glm.tract_names.copy()
tract_names.sort()

# Uncorrected
dwiplot.plot_distance_traces( params, tract_names, alpha=alpha, stat_type='uncorrected' )

# FDR corrected
dwiplot.plot_distance_traces( params, tract_names, alpha=alpha, stat_type='fdr' )

# RFT corrected
dwiplot.plot_distance_traces( params, tract_names, alpha=alpha, stat_type='rft' )


In [ ]:
# Plot scatterplots & violin plots for GLM results

# Specify GLM
glm = 'glm-age-sex'

# Set up parameters
params_plot = {}
params_plot['glm'] = glm
params_plot['alpha'] = 0.05
params_plot['outlier_z'] = params_glm[glm]['outlier_z']
params_plot['stat_type'] = 'rft'
params_plot['axis_font'] = 40
params_plot['marker_size'] = 15
params_plot['line_width'] = 4
params_plot['ticklabel_font'] = 33
params_plot['title_font'] = 28
params_plot['legend_font'] = 40
params_plot['legend_scale'] = 3
params_plot['show_title'] = False
params_plot['show_legend'] = True
params_plot['dimensions'] = (25,20)

# Loop through tracts and call plot function for each
dwiplot.plot_glm_results_all( params_plot, my_glm, verbose=True )
    

In [ ]:
# Aggregate tracts into single volumes
my_glm.aggregate_tracts(prefix='tract_final_norm_bidir', op='max', verbose=True)


In [ ]:
# Aggregate tracts crossing corpus callosum
cc_img = '/Users/lpzatr/OneDrive - The University of Nottingham/synched/projects/atlases/corpus_callosum_midline.nii'
my_glm.aggregate_tracts(prefix='tract_final_norm_bidir', suffix='_cc', intersects=cc_img, \
                        op='max', threshold=0, verbose=True)


In [ ]:
# Aggregate stats into single volumes

assert my_glm.aggregate_stats( op='max', pthres=0.05, suffix='-rft', verbose=True, clobber=True )


In [ ]:
# Generate Pajek graphs

assert my_glm.create_pajek_graphs( suffix='-rft', edge_val='tsum', verbose=True, clobber=True )
